# Using RDF triples to analyze a corpus of documents

A first draft of using OpenAI's API to summarize an article into a table of triples (subject, predicate, object) that shows the various entities in the articles and how they relate to one another.

In [22]:
import os
from io import StringIO
import advertools as adv
import adviz
import pandas as pd
pd.options.display.max_columns = None
from openai import OpenAI
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

## Get the XML sitemap of a website `foreignaffairs.com`

In [3]:
foraff_sitemap = adv.sitemap_to_df('https://www.foreignaffairs.com/robots.txt')
foraff_sitemap.head()

,loc,lastmod,changefreq,priority,sitemap,etag,sitemap_last_modified,sitemap_size_mb,download_date
0,https://www.foreignaffairs.com/articles/middle-east/2011-08-22/iraqs-relentless-insurgency,2023-07-20 17:07:46+00:00,weekly,1.0,https://www.foreignaffairs.com/sitemap.xml?page=2,"W/""1708078918""",2024-02-16 10:21:58,0.336972,2024-02-16 10:48:26.128358+00:00
1,https://www.foreignaffairs.com/articles/persian-gulf/2011-09-07/pushing-reform-bahrain,2023-07-13 18:37:25+00:00,weekly,1.0,https://www.foreignaffairs.com/sitemap.xml?page=2,"W/""1708078918""",2024-02-16 10:21:58,0.336972,2024-02-16 10:48:26.128358+00:00
2,https://www.foreignaffairs.com/articles/africa/2011-09-09/battle-nigerias-boko-haram-put-down-your-guns,2023-01-23 17:18:54+00:00,weekly,1.0,https://www.foreignaffairs.com/sitemap.xml?page=2,"W/""1708078918""",2024-02-16 10:21:58,0.336972,2024-02-16 10:48:26.128358+00:00
3,https://www.foreignaffairs.com/articles/benin/2011-07-27/hezbollah-party-fraud,2024-02-08 20:32:35+00:00,weekly,1.0,https://www.foreignaffairs.com/sitemap.xml?page=2,"W/""1708078918""",2024-02-16 10:21:58,0.336972,2024-02-16 10:48:26.128358+00:00
4,https://www.foreignaffairs.com/articles/middle-east/2011-09-16/why-israel-should-vote-palestinian-independence,2023-10-09 18:11:23+00:00,weekly,1.0,https://www.foreignaffairs.com/sitemap.xml?page=2,"W/""1708078918""",2024-02-16 10:21:58,0.336972,2024-02-16 10:48:26.128358+00:00


## Select any subset of interesting URLs
Articles about the USA published in the years 2021 - 2022

In [35]:
foraff_urllist = (foraff_sitemap
                  [foraff_sitemap['loc'].str.contains('/united-states/') &
                  foraff_sitemap['loc'].str.contains('202[1-2]')]
                  .sort_values('loc')
                  .tail(50)
                  ['loc'])
pd.concat([foraff_urllist[:5], foraff_urllist[-5:]])

34379                    https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first
34381                                   https://www.foreignaffairs.com/articles/united-states/2021-09-30/america-humble
34385                    https://www.foreignaffairs.com/articles/united-states/2021-10-04/covid-communication-breakdown
34384                          https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school
34387               https://www.foreignaffairs.com/articles/united-states/2021-10-05/fight-against-chinas-bribe-machine
34817    https://www.foreignaffairs.com/articles/united-states/2022-06-24/latin-americas-lessons-post-roe-united-states
34833                           https://www.foreignaffairs.com/articles/united-states/2022-07-08/americas-epidemic-hate
34834                                  https://www.foreignaffairs.com/articles/united-states/2022-07-11/end-magic-money
34836            https://www.foreignaffa

## Crawl the URLs and extract article text

In [5]:
adv.crawl(
    foraff_urllist,
    'foreignaffairs_usa_2021_2022.jl',
    xpath_selectors={
        'article_text': '//div[@class="article-dropcap--inner paywall-content"]//p/text()'
    },
    custom_settings={'LOG_FILE': 'foreignaffairs_usa_2021_2022.log'}
)

In [104]:
crawldf = pd.read_json('foreignaffairs_usa_2021_2022.jl', lines=True)
crawldf.head(2)

,url,title,meta_desc,viewport,charset,h1,h2,h3,h4,h5,canonical,alt_href,alt_hreflang,og:site_name,og:type,og:url,og:title,og:description,og:image,og:image:url,og:image:secure_url,og:image:width,og:image:height,og:updated_time,og:locale,twitter:card,twitter:description,twitter:title,twitter:creator,twitter:creator:id,twitter:url,twitter:image:width,twitter:image:height,twitter:image,jsonld_@context,jsonld_@type,jsonld_itemListElement,jsonld_1_@context,jsonld_1_@graph,body_text,size,article_text,download_timeout,download_slot,download_latency,depth,status,links_url,links_text,links_nofollow,nav_links_url,nav_links_text,nav_links_nofollow,header_links_url,header_links_text,header_links_nofollow,footer_links_url,footer_links_text,footer_links_nofollow,img_src,img_alt,img_loading,img_width,img_height,img_srcset,img_sizes,ip_address,crawl_time,resp_headers_Date,resp_headers_Content-Type,resp_headers_Cf-Ray,resp_headers_Cf-Cache-Status,resp_headers_Access-Control-Allow-Origin,resp_headers_Age,resp_headers_Cache-Control,resp_headers_Content-Language,resp_headers_Expires,resp_headers_Last-Modified,resp_headers_Link,resp_headers_Strict-Transport-Security,resp_headers_Vary,resp_headers_Via,resp_headers_Access-Control-Allow-Headers,resp_headers_Access-Control-Allow-Methods,resp_headers_Access-Control-Max-Age,resp_headers_X-Cache,resp_headers_X-Cache-Hits,resp_headers_X-Content-Type-Options,resp_headers_X-Drupal-Dynamic-Cache,resp_headers_X-Frame-Options,resp_headers_X-Generator,resp_headers_X-Pantheon-Styx-Hostname,resp_headers_X-Served-By,resp_headers_X-Styx-Req-Id,resp_headers_X-Timer,resp_headers_Server,request_headers_Accept,request_headers_Accept-Language,request_headers_User-Agent,request_headers_Accept-Encoding,request_headers_Cookie
0,https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first,The Age of America First: Washington’s Flawed New Foreign Policy Consensus,"Donald Trump was supposed to be an aberration. At first glance, the foreign policy of U.S. President Joe Biden could...","width=device-width, initial-scale=1, shrink-to-fit=no",utf-8,The Age of America First,@@Washington’s Flawed New Foreign Policy Consensus@@Most-Read Articles,By Richard Haass@@THE SQUANDERING@@EXTREME COMPETITION@@American Nationalism@@WITHDRAWAL AT ANY COST@@QUESTIONS OF ...,Sections @@ Topics @@ Regions @@ Article Types @@ Archive @@ Contact @@ Follow Us@@Subscribe to Foreign Affairs to ...,on Foreign Affairs!@@ Get in-depth analysis delivered right to your inbox,https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first,https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first@@https://www.foreigna...,en,Foreign Affairs,Article,https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first,The Age of America First,Washington's flawed new foreign policy consensus.,https://cdn-live.foreignaffairs.com/sites/default/files/styles/social_share/public/images/2021/09/28/15.%20Richard%2...,https://cdn-live.foreignaffairs.com/sites/default/files/styles/social_share/public/images/2021/09/28/15.%20Richard%2...,https://cdn-live.foreignaffairs.com/sites/default/files/styles/social_share/public/images/2021/09/28/15.%20Richard%2...,1200,630,2023-10-04T11:56:44-04:00,en_US,summary_large_image,Washington's flawed new foreign policy consensus.,The Age of America First,@foreignaffairs,21114659,https://www.foreignaffairs.com/articles/united-states/2021-09-29/biden-trump-age-america-first,1200,630,https://cdn-live.foreignaffairs.com/sites/default/files/styles/social_share/public/images/2021/09/28/15.%20Richard%2...,https://schema.org,BreadcrumbList,"[{'@type': 'ListItem', 'position': 1, 'name': 'North America', 'item': 'https://www.foreignaffairs.com/regions/north...",https://schema.org,"[{'@type': 'NewsArticle', 'headline': 'The Age of America First: Washington’s Flawed New Foreign Policy Consensus', ...",Foreign Affairs Magazin

## Ask for RDF triples in CSV format for each articles using OpenAI's API

In [105]:
article_rdf_list = []
for url, article in crawldf.dropna(subset=['article_text'])[['url', 'article_text']].values:
    print(f'Getting: {url}', end='\r')
    article_rdf = client.chat.completions.create(
        model='gpt-4',
        temperature=0,
        seed=123,
        messages=[
            {"role": "system", "content": "You are a helpful assistant, and an expert in RDF ontologies."},
            {"role": "user", "content": f"""
Please create a table of RDF triples for the following article.
Provide the results in CSV format.
Provide lower case columns: subject, predicate, object

Article:


{article.replace('@@', '\n')}
"""}
        ])
    article_rdf_list.append((url, article_rdf.dict()))

## Combine responses and save to a parquet file

In [84]:
df = (pd.concat([
    pd.json_normalize([art[1] for art in article_rdf_list]),
    pd.json_normalize(pd.DataFrame([art[1] for art in article_rdf_list])['choices'].str[0])], axis=1)
 .drop('choices', axis=1)
 .to_parquet('faffairs_rdf_responses.parquet', index=False, version='2.6')
)



In [ ]:
rdf_df = pd.read_parquet('faffairs_rdf_responses.parquet')

## Get responses that can be read into a CSV
Many responses are not consistent in their format, and can easily be re-formated to be read into a table.

Some work needs to be done in the prompt to ensure a consitent response format (consider a finetuned model also)

In [ ]:
rdf_df_list = []
rdf_df_list_to_clean = []
errors = []
for i, message in enumerate(rdf_df['message.content']):
    try:
        rdf_df = pd.read_csv(StringIO(message))
        if rdf_df.columns.tolist() == ['subject', 'predicate', 'object']:
            rdf_df_list.append(rdf_df.assign(index=i))
        else:
            rdf_df_list_to_clean.append((i, message))
    except Exception as e:
        errors.append((i, str(e)))

## Merge table into one DataFrame and combine with URls and article titles

In [92]:
rdf_all = pd.merge(
    pd.concat(rdf_df_list),
    crawldf.reset_index()[['index', 'url', 'h1']],
    left_on='index',
    right_on='index',
    how='left')



In [107]:
rdf_all.head()

,subject,predicate,object,index,url,h1
0,Inequality,is growing,NaN,1,https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school,The Trouble With Business School
1,Democracy,is under threat,NaN,1,https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school,The Trouble With Business School
2,Climate change,looms,NaN,1,https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school,The Trouble With Business School
3,COVID-19 pandemic,has exposed,massive inequality in health care,1,https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school,The Trouble With Business School
4,COVID-19 pandemic,has exposed,massive inequality in food security,1,https://www.foreignaffairs.com/articles/united-states/2021-10-04/trouble-business-school,The Trouble With Business School


## Which subjects were most discussed in these articles?

In [108]:
adviz.value_counts_plus(rdf_all['subject'], show_top=20, name='subject')

,subject,count,cum. count,%,cum. %
1,United States,114,114,33.1%,33.1%
2,European Union,25,139,7.3%,40.4%
3,Business schools,20,159,5.8%,46.2%
4,Antony Blinken,12,171,3.5%,49.7%
5,China,11,182,3.2%,52.9%
6,economic power,10,192,2.9%,55.8%
7,Xi,8,200,2.3%,58.1%
8,Cold War,7,207,2.0%,60.2%
9,Kassym-Jomart Tokayev,7,214,2.0%,62.2%
10,economic sanctions,5,219,1.5%,63.7%


## Which objects were most discussed in these articles?

In [109]:
adviz.value_counts_plus(rdf_all['object'], show_top=20)

,data,count,cum. count,%,cum. %
1,United States,12,12,3.5%,3.5%
2,European Union,7,19,2.0%,5.5%
3,Donald Trump administration,4,23,1.2%,6.7%
4,China,4,27,1.2%,7.8%
5,nan,3,30,0.9%,8.7%
6,2020,2,32,0.6%,9.3%
7,Boeing and Airbus,2,34,0.6%,9.9%
8,Johnson & Johnson,2,36,0.6%,10.5%
9,Transatlantic Trade and Investment Partnership,2,38,0.6%,11.0%
10,Russia,2,40,0.6%,11.6%


## Which subject/object pairs were most frequently discussed?

In [114]:
rdf_all[['subject', 'object']].value_counts().reset_index().head(10)

,subject,object,count
0,European Union,United States,11
1,United States,European Union,7
2,United States,Joe Biden,2
3,European Union,Donald Trump administration,2
4,United States,Donald Trump administration,2
5,European Union,China,2
6,United States,Pfizer-BioNTech,1
7,United States,Pittsburgh,1
8,United States,Russian intelligence agencies,1
9,United States,Russian intelligence officers operating under diplomatic cover,1


## How were the top subjects related to objects?

In [113]:
rdf_all[['subject', 'predicate']].value_counts().reset_index().head(20)

,subject,predicate,count
0,United States,may exploit,29
1,United States,needs to,9
2,Antony Blinken,discussedAbout,5
3,United States,faces,5
4,Antony Blinken,mentionedInDiscussion,4
5,United States,technology giant,4
6,United States,COVID-19 vaccine provider,3
7,building resilience,requires,3
8,Cold War,involved,3
9,Cold War,issues,3
